In [1]:
# import necessary packages
import skimage
from skimage import measure
import pandas as pd
import numpy as np
import os, sys
import argparse
import time
import cv2
import math
import tqdm.notebook as tqdm
import matplotlib
import matplotlib.pyplot as plt
import subprocess

from multiprocessing import Pool
from PIL import Image
from importlib import reload
from DGXutils import GetFileNames, GetLowestGPU

sys.path.append('../')

# custom

import utils.Traits as Traits

In [11]:
# get average areas per genotype
measurements["genotype"] = [name[:-6] for name in measurements["image_name"]]

groupby = measurements.groupby("genotype")
avg_measurements = pd.DataFrame({'avg_wing_area': groupby["wing area"].mean(),
                                 'avg_env_area': groupby["env area"].mean(),
                                 'avg_seed_area': groupby["seed area"].mean(),
                                 'avg_seed_count': groupby["seed_count"].mean(),
                                 'genotype': groupby["genotype"].first()})
avg_measurements.head()

,avg_wing_area,avg_env_area,avg_seed_area,avg_seed_count,genotype
genotype,,,,,
pred_1.tt8t x fc6-1.B.WBP-2022.W,1.035435,0.331851,0.155475,9.2,pred_1.tt8t x fc6-1.B.WBP-2022.W
pred_10.tt8t x fc5-2.B.WBP-2022.W,1.161073,0.400954,0.147018,9.3,pred_10.tt8t x fc5-2.B.WBP-2022.W
pred_100.ISU397_A.B.ISU Wild.S,0.856427,0.280958,0.101233,7.7,pred_100.ISU397_A.B.ISU Wild.S
pred_101.ISU398_A.B.ISU Wild.W,0.623864,0.342791,0.114140,7.5,pred_101.ISU398_A.B.ISU Wild.W
pred_102.ISU399_A.B.ISU Wild.W,0.857869,0.297062,0.119175,8.0,pred_102.ISU399_A.B.ISU Wild.W


In [45]:
# write a function to calculate ratio features between two classes
def between_ratio(mask, feature1: str, feature2: str, type: str = "area"):

    """
    Calculates the ratio of a feature between two classes.

    Parameters:
        mask (np.array): A 3-channel mask with range 0-255.
        feature1 (str): The first feature to calculate the ratio between.
        feature2 (str): The second feature to calculate the ratio between.
        type (str): The type of feature to calculate the ratio of. Must be one of 'area' or 'perimeter.'
    
    Returns:
        Ratio (float): The ratio of the feature between the two classes. Calculated as feature1 / feature2.
    """

    # make sure the strings passed are valid
    assert feature1 and feature2 in ["wing", "env", "seed"], "Features must be one of 'wing', 'env', or 'seed'."
    assert type in ["area", "perimeter"], "Type must be one of 'area' or 'perimeter'."
    
    

    if type == "area":

        # separate out the classes
        wing = mask[:,:,0] > 128
        env = mask[:,:,1] > 128
        seed = mask[:,:,2] > 128

        # get areas by class
        wing_area, env_area, seed_area = wing.sum(), env.sum(), seed.sum()

        # make dict to map strings to indices
        feature_map = {"wing": wing_area, 
                       "env": env_area, 
                       "seed": seed_area}

        # get ratio of feature between two classes
        return feature_map[feature1] / feature_map[feature2]

    elif type == "perimeter":
        
        # get perimeters by class
        wing_perimeter, env_perimeter, seed_perimeter = perimeter(mask)

        # make dict to map strings to indices
        feature_map = {"wing": wing_perimeter, 
                       "env": env_perimeter, 
                       "seed": seed_perimeter,}

        # get ratio of feature between two classes
        return feature_map[feature1] / feature_map[feature2]
